## Agreements

In [1]:
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [6]:
from django.db.models import Count

users = []
for agg in ArticleLabel.objects.values('user__username').annotate(Count('user__username')):
     users.append(agg['user__username'])   
users.remove("jmperez")
users

['asilva', 'fdesanctis', 'gdamill', 'lprats']

In [7]:


base_query = {
    "user__username__in": users
}
ArticleLabel.objects.filter(**base_query).count()

20

¿Cuántos etiquetamos todes?

In [8]:
import pandas as pd

pd.options.display.max_columns = 40


articles_labelled = {label.article_id for label in ArticleLabel.objects.filter(**base_query)}

df = pd.DataFrame({"name":users})
df.set_index("name", inplace=True)

for label in ArticleLabel.objects.filter(**base_query):
    username = label.user.username
    df.loc[username, label.article.title] = int(label.is_interesting)
    
df

,Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria”,Escándalo en España: fallan los tests de coronavirus fabricados en China,El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón,"Luis Novaresio contó cómo será su boda con Braulio Bauab: ""Nos va a casar una rabina""",Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad”
name,,,,,
asilva,1.0,1.0,1.0,1.0,1.0
fdesanctis,1.0,1.0,1.0,1.0,1.0
gdamill,1.0,1.0,1.0,1.0,1.0
lprats,1.0,1.0,1.0,1.0,1.0


## Agreements sobre odio

Ok, acá agarremos todos los labels sobre comentarios de todes

In [9]:
article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)

formatted_labels = []

df_comments = pd.DataFrame({"name": users})

df_comments.set_index("name", inplace=True)

for article_label in article_labels:
    username = article_label.user.username
    for comment_label in article_label.comment_labels.all():
        df_comments.loc[username, comment_label.comment_id] = comment_label.is_hateful

# Saco los que no estén etiquetados por todes
#df_comments = df_comments[df_comments.columns[df_comments.notna().all()]]

df_comments

,333495,333496,333497,333498,333499,333500,333501,333502,333503,333504,333505,333506,333507,333508,333509,333510,333511,333512,333513,333514,...,371947,371948,371949,371950,371951,371952,371953,371954,371955,371956,371957,371958,371959,371960,371961,371962,371963,371964,371965,371966
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
asilva,False,True,False,True,False,False,False,False,True,False,True,True,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False
fdesanctis,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,True,False,False,False,True,True,False,False,False,True,True,True,True
gdamill,False,True,False,True,False,False,True,False,True,False,True,True,False,False,False,False,False,True,True,False,...,True,True,False,False,False,False,True,True,False,False,False,False,True,True,True,False,True,True,True,True
lprats,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,True,False,True,False,True,False,True,True,False,True,True,True,True


In [10]:
from nltk import agreement

def kappa_of(df, subset=None):
    if subset:
        coders = df.loc[subset]
    else:
        coders = df
    
    coders = coders[coders.columns[coders.notna().all()]]
    
    coders = coders.values
    formatted_labels = []
    for i in range(coders.shape[0]):
        # Acá las ponemos siguiendo el siguiente formato
        # [num etiquetador, num instancia, label]
        for j in range(coders.shape[1]):
            formatted_labels.append((i, j, coders[i, j]))


    ratingtask = agreement.AnnotationTask(data=formatted_labels)

    return ratingtask.kappa(), coders.shape[1]


for subset in [
    users,
    ]:
    name = ' '.join(subset)
    kappa, instances = kappa_of(df_comments, subset)
    print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")
    
print("\n\nPares\n\n")

for i in range(len(users)):
    for j in range(i+1, len(users)):
        subset = [users[i], users[j]]
        name = ' '.join(subset)
        kappa, instances = kappa_of(df_comments, subset)
        print(f"Fleiss K para {name:<40} = {kappa:.3f} (#instancias = {instances})")


Fleiss K para asilva fdesanctis gdamill lprats         = 0.599 (#instancias = 241)


Pares


Fleiss K para asilva fdesanctis                        = 0.598 (#instancias = 241)
Fleiss K para asilva gdamill                           = 0.532 (#instancias = 241)
Fleiss K para asilva lprats                            = 0.546 (#instancias = 241)
Fleiss K para fdesanctis gdamill                       = 0.588 (#instancias = 241)
Fleiss K para fdesanctis lprats                        = 0.661 (#instancias = 241)
Fleiss K para gdamill lprats                           = 0.667 (#instancias = 241)


In [11]:
print("Cantidad de mensajes etiquetados como odiosos")
df_comments.sum(axis=1)

Cantidad de mensajes etiquetados como odiosos


name
asilva         84.0
fdesanctis    100.0
gdamill       139.0
lprats        123.0
dtype: float64

## Agreements sobre cada categoría

In [12]:
idx = pd.MultiIndex.from_product([CommentLabel.type_mapping, users],
                                names=["categoria", "etiquetador"])

df_labels = pd.DataFrame(index=idx)

df_labels

Empty DataFrame
Columns: []
Index: [(MUJER, asilva), (MUJER, fdesanctis), (MUJER, gdamill), (MUJER, lprats), (LGBTI, asilva), (LGBTI, fdesanctis), (LGBTI, gdamill), (LGBTI, lprats), (RACISMO, asilva), (RACISMO, fdesanctis), (RACISMO, gdamill), (RACISMO, lprats), (POBREZA, asilva), (POBREZA, fdesanctis), (POBREZA, gdamill), (POBREZA, lprats), (POLITICA, asilva), (POLITICA, fdesanctis), (POLITICA, gdamill), (POLITICA, lprats), (DISCAPACIDAD, asilva), (DISCAPACIDAD, fdesanctis), (DISCAPACIDAD, gdamill), (DISCAPACIDAD, lprats), (ASPECTO, asilva), (ASPECTO, fdesanctis), (ASPECTO, gdamill), (ASPECTO, lprats), (CRIMINAL, asilva), (CRIMINAL, fdesanctis), (CRIMINAL, gdamill), (CRIMINAL, lprats), (OTROS, asilva), (OTROS, fdesanctis), (OTROS, gdamill), (OTROS, lprats)]

In [14]:

article_labels = ArticleLabel.objects.filter(
    user__username__in=users,
)


for article_label in article_labels:
    username = article_label.user.username
    for comment_label in article_label.comment_labels.all():
        for name, field_name in CommentLabel.type_mapping.items():
            df_labels.loc[(name, username), comment_label.comment_id] = getattr(comment_label, field_name)

#df_labels

In [15]:
print("##", ' - '.join(users), '\n')

#users = ["mkondra", "jmperez"]

for category in CommentLabel.type_mapping:
    try:
        kappa, instances = kappa_of(df_labels.loc[category].loc[users])
        print(f"| {category:<12}   |  {kappa:.3f}")
    except ZeroDivisionError as e:
        print(f"No hay elementos para calcular Kappa de {category} - salteando")

print(f"Instancias: {instances}")


## asilva - fdesanctis - gdamill - lprats 

| MUJER          |  0.507
| LGBTI          |  0.758
| RACISMO        |  0.604
No hay elementos para calcular Kappa de POBREZA - salteando
| POLITICA       |  0.333
| DISCAPACIDAD   |  0.349
| ASPECTO        |  0.455
| CRIMINAL       |  0.938
No hay elementos para calcular Kappa de OTROS - salteando
Instancias: 241


## Alpha Krippendorff

In [51]:
import krippendorff

users = list(df_labels.loc["MUJER"].index)

user_combinations = [users]
user_combinations += [[u1, u2] for u1, u2 in set((u1, u2) for u1 in users for u2 in users if u1 != u2 and u1 < u2)]

for user_list in user_combinations:
    print("="*80 + '\n' * 2)
    print(" - ".join(user_list), "\n")
    for category in CommentLabel.type_mapping:
        mat = df_labels.loc[category].loc[user_list].values
        mat = mat.astype(float)
        alpha = krippendorff.alpha(mat)
        print(f"| {category:<12}   |  {alpha:.3f}")





asilva - fdesanctis - gdamill - lprats 

| MUJER          |  0.505
| LGBTI          |  0.758
| RACISMO        |  0.620
| POBREZA        |  0.333
| POLITICA       |  0.349
| DISCAPACIDAD   |  0.438
| ASPECTO        |  0.457
| CRIMINAL       |  0.938
| OTROS          |  -0.002


fdesanctis - lprats 

| MUJER          |  0.662
| LGBTI          |  0.829
| RACISMO        |  0.525
| POBREZA        |  nan
| POLITICA       |  0.389
| DISCAPACIDAD   |  -0.008
| ASPECTO        |  0.539
| CRIMINAL       |  0.957
| OTROS          |  nan


asilva - lprats 

| MUJER          |  0.412
| LGBTI          |  0.700
| RACISMO        |  0.705
| POBREZA        |  0.000
| POLITICA       |  -0.015
| DISCAPACIDAD   |  -0.004
| ASPECTO        |  0.354
| CRIMINAL       |  0.956
| OTROS          |  -0.004


asilva - fdesanctis 

| MUJER          |  0.521
| LGBTI          |  0.787
| RACISMO        |  0.519
| POBREZA        |  0.000
| POLITICA       |  0.497
| DISCAPACIDAD   |  0.663
| ASPECTO        |  0.255
| CR




asilva - fdesanctis - gdamill - lprats 

| Clase          | Acuerdo | 
|----------------|---------|
| MUJER          |  0.505  |
| LGBTI          |  0.758  |
| RACISMO        |  0.620  |
| POBREZA        |  0.333  |
| POLITICA       |  0.349  |
| DISCAPACIDAD   |  0.438  |
| ASPECTO        |  0.457  |
| CRIMINAL       |  0.938  |
| OTROS          |  -0.002 |



Apreciaciones generales:

- CRIMINAL: altísimo acuerdo
- MUJER: acuerdo razonable, pero hay que mejorar
- LGBTI y RACISMO: acuerdo relativamente alto
- POBREZA: ver qué pasa acá (ver Gabriel y Alan que etiquetaron lo mismo)
- POLITICA: esperable bajo acuerdo
- ASPECTO: ver acá qué pasó
- DISCAPACIDAD: ver también
- OTROS: ¿qué marcaron?




## MUJER


In [60]:
from collections import defaultdict

def no_null_columns(df):
    return df[df.columns[df.notna().all()]]

            
def show_from_df(df, show=False):
    article_to_comments = defaultdict(list)
    
    for comment_id in df.columns:
        comment = Comment.objects.get(id=comment_id)
        article_to_comments[comment.article_id].append(comment)
        
    for article_id, comments in article_to_comments.items():
        print("="*45 + "\n")
        article = comments[0].article
        print(article.title, "\n")
        
        for comment in comments:
            print("  - ",comment.text, end="")
            if show:
                yes = [annotator for annotator, v in df[comment.id].items() if v] 
                no = [annotator for annotator, v in df[comment.id].items() if not v] 
                print(f"\n     - SI: {yes}  NO: {no}")
            else:
                print("")

In [61]:
def get_agrees_and_disagrees(df_labels, category, users):    
    category_df = df_labels.loc[category].loc[users]

    category_df = no_null_columns(category_df)
    agree_comments = category_df[category_df.columns[category_df.all()]]
    non_hateful_comments = category_df[category_df.columns[(~category_df.astype(bool)).all()]]
    no_agree = ~((category_df.sum() == 0) | (category_df.sum() == len(users)))
    
    disagree_comments = category_df[category_df.columns[no_agree]]
    # Estos son muchos! Filtrar sólo los de aquellos artículos
    # donde haya habido algún acuerdo
    
    allowed_articles = set(
        c.article_id for c in Comment.objects.filter(id__in=agree_comments.columns)
    )
    
    allowed_comments = Comment.objects.filter(
        id__in=non_hateful_comments.columns,
        article_id__in=allowed_articles,
    )
    
    #print(len(allowed_comments))
    comment_ids = set(c.id for c in allowed_comments)
    
    non_hateful_comments = non_hateful_comments[[
        c for c in non_hateful_comments.columns if c in comment_ids
    ]]
    return agree_comments, disagree_comments, non_hateful_comments


agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "MUJER", users
)

In [62]:
show_from_df(agree_comments)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

  -  @usuario Hablo la catadora de pi........

Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 

  -  @usuario De los Presos que sacan de las carceles ni una Palabra mucho feminismo y no e visto a ninguna de estas aborteras de izquierda defender los derechos de las mujeres ni criticar a Zaffaroni que defiende y apoya criminales y violadores con su teoria abolicionista.
  -  @usuario Ponete Otowil tarada
  -  @usuario Pense que ya no le daban mas camara a esta imbecil, mentirosa
  -  @usuario Y tal vez no es culpable, vaya uno a saber,con el circo que hizo esta y lo demente que está la otra que lo denunció primero...
  -  @usuario No entiendo cómo le pueden creer si a cualquiera se le mete la pija para adentro con esta
  -  @usuario Esta pendeja usó a Juan una noche de tensión sexual y después es tan más agradecida en morderle la mano a quien le dio de com

Busquemos dónde hay discrepancias

In [63]:
show_from_df(disagree_comments, show=True)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

  -  @usuario Las cirugías le deformaron la cara.
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario Que la chupe
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario Para decir algo público al menos deberia quitarse la máscara; así la reconocemos y sabemos seguro que fue ella quien lo dijo. Y no alguna nieta perdida, tal vez.
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario caer en manos de cualquier cirujano plastico  tmbn ...
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario Cuidado con esta chica que sabe mucho de medicina.
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario Y vos me parecés una pelotuda a rosca
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario Quedate en España con Almodovar como hiciste casi toda tu vida...
     - SI: ['gda

## Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

> - @usuario Las cirugías le deformaron la cara.
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
> - @usuario Para decir algo público al menos deberia quitarse la máscara; así la reconocemos y sabemos seguro que fue ella quien lo dijo. Y no alguna nieta perdida, tal vez.
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
> - @usuario caer en manos de cualquier cirujano plastico  tmbn ...
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
> - @usuario Irresponsable es el cirujano que te operó la cara 🧟‍♀️
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']


#### Apreciaciones sobre el aspecto (hasta cierto punto) no los consideramos acá*


> - @usuario Cuidado con esta chica que sabe mucho de medicina.
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
> - @usuario Y vos me parecés una pelotuda a rosca
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
> - @usuario Que la chupe
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
> - @usuario Y a quien le importa la opinión de esta mujer...! a mi me pareció fantástica y cuál es la diferencia?
>   - SI: ['gdamill', 'lprats']  NO: ['asilva', 'fdesanctis']
> - @usuario Realmente a quien mierda le importa lo que diga esta???
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']

#### Agresiones pero no discriminatorias

> - @usuario Quedate en España con Almodovar como hiciste casi toda tu vida...
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats]
> - @usuario Cecilia shhhh...no tenes altura moral para hacer comentarios de ninguna índole
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
> - @usuario Me importa un choto lo que diga esta estúpida
>   - SI: ['asilva', 'gdamill']  NO: ['fdesanctis', 'lprats']
> - @usuario En qué te afectó? Si estabas en casa , qué te importa!!! Estás en edad de riesgo así que cuídate, quédate en casa que Alberto y Cristina te cuidan. Mira que cuando hablas boludeces, entra por la boca.
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']


#### Este es discriminatorio

> - @usuario Irresponsable es q te sigas falopeando a los 60 años, vieja trola.
>    - SI: ['asilva', 'fdesanctis', 'gdamill']  NO: ['lprats']


#### No discriminatorio al menos por mujer

> - @usuario Esta vieja está en pedo
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
> - @usuario Esta momia vive en Argentina o España?
>   - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']


> @usuario Irresponsabilidad sanitaria, es revolcarse con "feto" paez, ridicula!!!
> SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
> @usuario Comerse a Fito es anti sanitario
> SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']

     
=============================================

## Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

>  -  @usuario Y claro se llenó d comentarios de KKs a favor d la yegua mal parida y d culos rotos q defienden a otros culos rotos.. bueno o sea la yegua con la ley favoreció al país para q los maracas y lesbianas se casen, y eso la hace una gran persona, jajajaja mientras t chorea todo!!!
>    -  SI: ['gdamill', 'lprats']  NO: ['asilva', 'fdesanctis']

Polémica. Yo creo que sí, en este caso habla de "culos rotos" en el contexto de una nota LGBTI

>  -  @usuario La rabina ,si es que eso existe,deberá estudiar en el Toráh la opinión que tiene yaveh acerca de la sodomía.....hagan lo que se les cante pero no sé metan con creencias milenarias.....dios no ama a los promiscuos
>   -     SI: ['lprats']  NO: ['asilva', 'fdesanctis', 'gdamill']

Está border eh. Yo hubiera puesto que sí.


## Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 

=============================================

## Thelma Fardin contó cómo sigue la causa contra Juan Darthés: “Brasil le permite la impunidad” 


#### No son discriminatorios

>  -  @usuario No te salió el negocio amigue
>     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
>  -  @usuario Deja de ensusiar a la gente
>     - SI: ['fdesanctis']  NO: ['asilva', 'gdamill', 'lprats']
>  -  @usuario Y vos también sos impune deja de boludear
>     - SI: ['fdesanctis', 'lprats']  NO: ['asilva', 'gdamill']
>  -  @usuario Argentina a vos también
>     - SI: ['fdesanctis', 'lprats']  NO: ['asilva', 'gdamill']


  -  @usuario presa por mentirosa.
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario Fardin deja la merca,te hace mal. le cagaste la vida a un tipo para tener cámara y te cagaste en las víctimas reales.
     - SI: ['asilva', 'gdamill', 'lprats']  NO: ['fdesanctis']
  -  @usuario Y en Argentina también. Si no ya estarías imputada por violación a los derechos de autor.
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'fdesanctis']
  -  @usuario Che Fardín, Andate a robar fotos y dejate de romper las pelotas!!!
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'fdesanctis']
  -  @usuario Brasil está en el medio de una pandemia deja de hinchar las pelotas y ponete a tejer calzones en tu casa
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario Es lo único que puede decir la tontita esta? Ya la prensa debería sacarle los microfonos no aporta nada ni va a aportar. Y la boba no se da cuenta de que la usan con fines politicos. He ahi la demostración de su escaso intelecto
     - SI: ['asilva', 'gdamill', 'lprats']  NO: ['fdesanctis']
  -  @usuario Otra que me da vergüenza ajena
     - SI: ['lprats']  NO: ['asilva', 'fdesanctis', 'gdamill']
  -  @usuario Terminala flaca, no existís. Te mandaste ese chamuyo para que te conociéramos y las aborteras te llevaran de bandera, pero tus 15 minutos de fama se acabaron. Proba con buscar un empleo honesto ahora, mamu. Aburriste! 😤
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario Y Argentina te permite seguir siendo actriz. Bodriooooooo
     - SI: ['lprats']  NO: ['asilva', 'fdesanctis', 'gdamill']
  -  @usuario Ya no le importas a nadie idiota, te usaron como a un forro nena
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']

  -  @usuario NENA NO TENES DE K HABLAR. CON EL MOMENTO QUE VIVIMOS YA SABEMOS Y LO SIENTO UN HDP PERO ERAS YA GRANFECITA LO PODÍAS DENUNCIAR PERO NO QUERIAS QUEDARTE SIN TRABAJO LA PIFIASTE PORK LE HAVIAS JUIVIO AL CANAL TAMBIEN
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'fdesanctis']
  -  @usuario No hay otra noticia??????
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario Insufrible la minita esa... Ya no sabe cómo seguir currando...
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'fdesanctis']
  -  @usuario Quien se acuerda de la mina ésta?
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario Y vos seguís envuelta en tu mentira,creíste que ibas a salirte con la tuya, payasa!
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario NO SABE CÓMO SACAR MÁS PUBLICIDAD DE ESTO, SEA VERDAD O MENTIRA.
     - SI: ['lprats']  NO: ['asilva', 'fdesanctis', 'gdamill']
  -  @usuario Que mentiras dice ahora?
     - SI: ['fdesanctis', 'lprats']  NO: ['asilva', 'gdamill']
  -  @usuario Volvé Juan que acá te liberan y más cuando la denuncia es falsa.
     - SI: ['fdesanctis', 'gdamill']  NO: ['asilva', 'lprats']
  -  @usuario Queres la reconstruccion del hecho  pero con vos de protagonista ....o no???
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'fdesanctis']
  -  @usuario Porque no prueba en buscar trabajo y mientras tanto sigue la causa????
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'fdesanctis']
  -  @usuario O fue todo una escandalosa puesta en escena.
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario Por todas las cosas raras q se manda ya ni le creo
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario Volvé Juan que acá te liberan y más aún cuando la denuncia es falsa.
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']

In [64]:
%%capture
show_from_df(non_hateful_comments)

## LGBTI


In [65]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "LGBTI", users
)

In [66]:
show_from_df(agree_comments)


Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

  -  @usuario Y claro se llenó d comentarios de KKs a favor d la yegua mal parida y d culos rotos q defienden a otros culos rotos.. bueno o sea la yegua con la ley favoreció al país para q los maracas y lesbianas se casen, y eso la hace una gran persona, jajajaja mientras t chorea todo!!!
  -  @usuario asco
  -  @usuario 2 trolos casados por una rabina y escoltados por 2 pavorreales con la camiseta de river
  -  @usuario 🤢
  -  @usuario Aféitense, chicas! 💋
  -  @usuario ups el barba era trolis ... ups y parecía gente de bien
  -  @usuario Asco
  -  @usuario Que par de putos.
  -  @usuario Asco
  -  @usuario La pucha, parece que son más los anormales. 
Hacia donde va la humanidad?
Que dos anormales se juntan, es noticia en un diario.
  -  @usuario Entre el covid y esta nota..es igual . Ambos  causan desagrado🙄
  -  @usuario Y a quién joraca le importa q estos 2 putitos se casen, encima se sacan sel

In [67]:
show_from_df(disagree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 

  -  @usuario Y la culp es de españa ?  O los putos chinos ?
     - SI: ['lprats']  NO: ['asilva', 'fdesanctis', 'gdamill']

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

  -  @usuario Me chupa un hue...no, mejor no.
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario Que cosa peluda 🦍😎
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario En ningun parrafo de los 70 libros que fueron escritos por inspiracion divina y en estos libros sagrados solamente figuran UNGIDOS los varones para el sacerdocio...!?
     - SI: ['lprats']  NO: ['asilva', 'fdesanctis', 'gdamill']
  -  @usuario De postre Banana... en todas sus presentaciones... 
En ese casorio tiras una banana y no toca el piso... 
Tendran de que colgarse los invitados?
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario Tal vez después de eso se 

In [69]:
%%capture
show_from_df(non_hateful_comments)

## RACISMO

In [70]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "RACISMO", users
)

In [71]:
show_from_df(agree_comments)


Escándalo en España: fallan los tests de coronavirus fabricados en China 

  -  @usuario Y la culp es de españa ?  O los putos chinos ?
  -  @usuario Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
Chinos del orto
  -  @usuario CHINOS DE MIERDA
OJALÁ SE EXTINGAN
  -  @usuario Estos chinos son una mierda


In [72]:
show_from_df(disagree_comments, show=True)


Escándalo en España: fallan los tests de coronavirus fabricados en China 

  -  @usuario MADE IN CHINA....que raro que falle che -_-
     - SI: ['asilva', 'gdamill', 'lprats']  NO: ['fdesanctis']
  -  @usuario @usuario Es chino es basura.

Los chinos compraron mascaras fuera de China, para proteger a su personal sanitario, por algo seria que no compararon las que ellos mismo fabrican o no las usaron.

China es una nación muy linda, pero produce basura, no tiene calidad en sus productos
     - SI: ['asilva', 'gdamill', 'lprats']  NO: ['fdesanctis']
  -  @usuario Los chinos van por todos nosotros.Necesitan espacio !!!
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario @usuario No entendemos que lo CHINO no sirve🙄
     - SI: ['asilva', 'gdamill', 'lprats']  NO: ['fdesanctis']
  -  @usuario Chinverguenzas
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario Pero que se puede esperar de comunistas y de asiaticos
     - SI: ['fdesanctis', 'g

In [74]:
%%capture
show_from_df(non_hateful_comments)

## POBREZA

In [75]:

agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "POBREZA", users
)

In [76]:
show_from_df(agree_comments)

In [77]:
show_from_df(disagree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

  -  @usuario Que asco esos negros imundos lacras parasitos abria que bañarlos en corona virus
     - SI: ['asilva', 'gdamill']  NO: ['fdesanctis', 'lprats']


In [78]:
show_from_df(non_hateful_comments)

## POLITICA

In [79]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "POLITICA", users
)

In [80]:

show_from_df(agree_comments)

In [82]:
show_from_df(disagree_comments, show=True)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

  -  @usuario Kuka
     - SI: ['asilva', 'fdesanctis', 'gdamill']  NO: ['lprats']
  -  @usuario Mingo... Mira quien habla? Otra rata salió de su cueva.... OTRA VENDE PATRIA.... POR DIOS... CUANTOS TRAIDORES QUE HAY EN NUESTRO QUERIDA PATRIA 🇦🇷🇦🇷🇦🇷🇦🇷
     - SI: ['lprats']  NO: ['asilva', 'fdesanctis', 'gdamill']
  -  @usuario Facho
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']

Escándalo en España: fallan los tests de coronavirus fabricados en China 

  -  @usuario Pero que se puede esperar de comunistas y de asiaticos
     - SI: ['gdamill', 'lprats']  NO: ['asilva', 'fdesanctis']
  -  @usuario China te manda el virus
Te cobra el test...no anda
El partido comuns=ista china si sibrevife esto...nos gobernara a todos...
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']

Luis Novaresio contó cómo será su boda con Braulio Bauab: "Nos va a casar una rabina" 

  -  @usuario Y cl

In [83]:
%%capture
show_from_df(non_hateful_comments)

## ASPECTO

In [84]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "ASPECTO", users
)

In [85]:
show_from_df(agree_comments)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

  -  @usuario Irresponsable es q te sigas falopeando a los 60 años, vieja trola.
  -  @usuario Esta momia vive en Argentina o España?


In [86]:
show_from_df(disagree_comments, show=True)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

  -  @usuario Las cirugías le deformaron la cara.
     - SI: ['asilva', 'gdamill', 'lprats']  NO: ['fdesanctis']
  -  @usuario Estamos grandes???no la conocí....
     - SI: ['lprats']  NO: ['asilva', 'fdesanctis', 'gdamill']
  -  @usuario Para decir algo público al menos deberia quitarse la máscara; así la reconocemos y sabemos seguro que fue ella quien lo dijo. Y no alguna nieta perdida, tal vez.
     - SI: ['asilva', 'lprats']  NO: ['fdesanctis', 'gdamill']
  -  @usuario caer en manos de cualquier cirujano plastico  tmbn ...
     - SI: ['asilva', 'gdamill']  NO: ['fdesanctis', 'lprats']
  -  @usuario Irresponsable es el cirujano que te operó la cara 🧟‍♀️
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario Esta vieja está en pedo
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario 🤮🤮🤮🤮
     - SI: ['fdesanctis']  NO: ['asilva', 'gdamill', 'lprats']
  

In [87]:
show_from_df(non_hateful_comments)


Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

  -  @usuario 1ro q se informe xq la marcha no fue anti cuarentena, así q habla al pedo 💨
  -  @usuario Y a quien carajo le importa lo que diga. A mi no me representa en nada.
  -  @usuario Kuka
  -  @usuario Me importa un tereso la opinión de ésta gente.
  -  @usuario Que la chupe
  -  @usuario que se calle 
es cómplice del saqueo
hijos de puta
  -  @usuario Francamente, a quié le puede importar lo que opinás!!!!
  -  @usuario Pasarte de merca durante años, es una verdadera irresponsabilidad sanitaria
  -  @usuario Sos muy suave en el parecer con los asistentes ,yo diría que fue una demostración de desprecio para todo el personal de salud , médicos,enfermeros , fallecidos y enfermos a los que paradojicamente van a tener que recurrir poniendo caritas de bueno y negando en caso de enfermarse
  -  @usuario Acepten al que piensa distinto.
  -  @usuario Y a quien le importa lo q diga Cecilia Roth!!
  -  

## CRIMINAL

In [88]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "CRIMINAL", users
)

In [89]:
show_from_df(agree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

  -  @usuario SIMPLE y SENCILLAMENTE  1 -
     - SI: ['asilva', 'fdesanctis', 'gdamill', 'lprats']  NO: []
  -  @usuario Punto a favor
     - SI: ['asilva', 'fdesanctis', 'gdamill', 'lprats']  NO: []
  -  @usuario Uno menos.
     - SI: ['asilva', 'fdesanctis', 'gdamill', 'lprats']  NO: []
  -  @usuario no se perdió nada
     - SI: ['asilva', 'fdesanctis', 'gdamill', 'lprats']  NO: []
  -  @usuario Al fin una buena!!!
     - SI: ['asilva', 'fdesanctis', 'gdamill', 'lprats']  NO: []
  -  @usuario Uno menos..
     - SI: ['asilva', 'fdesanctis', 'gdamill', 'lprats']  NO: []
  -  @usuario 👏🏻👏🏻👏🏻 sigan sigan
     - SI: ['asilva', 'fdesanctis', 'gdamill', 'lprats']  NO: []
  -  @usuario Una pena terrible....
     - SI: ['asilva', 'fdesanctis', 'gdamill', 'lprats']  NO: []
  -  @usuario Bueno...
Que se perdió?
     - SI: ['asilva', 'fdesanctis', 'gdamill', 'lprats']  NO: []
  -  @usuario Igual no servía para n

In [90]:
show_from_df(disagree_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

  -  @usuario Pobre ciudadano, seguramente era inocente.
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario Mira lo que es ese muñeco por favor 💀
     - SI: ['asilva', 'gdamill', 'lprats']  NO: ['fdesanctis']
  -  @usuario Qepd ahre
     - SI: ['fdesanctis', 'gdamill']  NO: ['asilva', 'lprats']
  -  @usuario Q enorme perdida
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario Que lastima
     - SI: ['fdesanctis', 'gdamill']  NO: ['asilva', 'lprats']
  -  @usuario Solo Me preocupa por los gastos q ocasionaron. Hay q ir con todo
     - SI: ['fdesanctis', 'gdamill', 'lprats']  NO: ['asilva']
  -  @usuario Delicatesen los tatus
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']
  -  @usuario Bueno. Sigamos con cosas mas interesantes.
     - SI: ['gdamill']  NO: ['asilva', 'fdesanctis', 'lprats']


In [61]:
show_from_df(non_hateful_comments, show=True)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

@infobae y la verdad que mucha lastima no me da.. {'jmperez': False, 'mkondra': False}
@infobae LA TIA DE QUIEN 😒😒😒🙄 {'jmperez': False, 'mkondra': False}
@infobae Podrían experimentar con estos seres posibles mutaciones y vacunas así hacen su aporte para la sociedad {'jmperez': False, 'mkondra': False}
@infobae Qué pérdida! {'jmperez': False, 'mkondra': False}
@infobae No perdimos nada entonces, palo y a la bolsa!! {'jmperez': False, 'mkondra': False}
@infobae Igual no servía para nada...... {'jmperez': False, 'mkondra': False}
@infobae Subir la foto con un bebé no la hace menos culpable! {'jmperez': False, 'mkondra': False}
@infobae Pobre ciudadano, seguramente era inocente. {'jmperez': False, 'mkondra': False}
@infobae Papá...que caripela!!! Menos mal que no murió un santo....sino mañana hablan de mártir y lo hacen beato. {'jmperez': False, 'mkondra': False}
@infobae Una pena terrible.... {'jmperez':

In [51]:
show_from_df(non_hateful_comments)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

@infobae y la verdad que mucha lastima no me da..
@infobae LA TIA DE QUIEN 😒😒😒🙄
@infobae Podrían experimentar con estos seres posibles mutaciones y vacunas así hacen su aporte para la sociedad
@infobae Qué pérdida!
@infobae No perdimos nada entonces, palo y a la bolsa!!
@infobae Igual no servía para nada......
@infobae Subir la foto con un bebé no la hace menos culpable!
@infobae Pobre ciudadano, seguramente era inocente.
@infobae Papá...que caripela!!! Menos mal que no murió un santo....sino mañana hablan de mártir y lo hacen beato.
@infobae Una pena terrible....
@infobae Todavía no renunciaron Saín y la antropóloga?
@infobae Q enorme perdida
@infobae Juegue,  juegue
@infobae Ya estaba muerto ese muchacho!!! Le paso algo mejor ahora
@infobae Una sola palabra : KARMA
@infobae Qepd ahre
@infobae Mira lo que es ese muñeco por favor 💀
@infobae Delicatesen los tatus
@infobae La pena de estos delincuentes n

## SALUD


In [63]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "DISCAPACIDAD", users
)

In [65]:
show_from_df(agree_comments)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Nadie la juzga por su adicción a las drogas. Que no venga a criticar las expresiones de voluntad.
@LANACION Simplemente no me interesa lo que diga una adicta

Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

@infobae Vieja merquera
@infobae Irresponsabilidad sanitaria es vivir falopeada, Cecilia.

Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios” 

@clarincom Alejenle la botella por el bien de todos
@clarincom Por imagen él tendría que guardarse de por vida , borracho impresentable !!! Vergüenza mundial este payaso peroncho 💩
@clarincom Mire borracho ridículo, si es por imagen, deberíamos prohibirlo a ud!
@clarincom Al fin Ginés, va a quedar como un viejito bueno, solo que le gusta un poco el trago nomas?


In [66]:
show_from_df(disagree_comments, show=True)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Esta señora grupo de riesgo debería cuidarse de no excederse con la fafafa en vez de usar el dedito acusador. {'jmperez': False, 'mkondra': True}
@LANACION RECORDAS KUANDO VENDIAS JUNTO A FITO KILOS DE MERKA ???? {'jmperez': False, 'mkondra': True}

El proyecto del impuesto a la riqueza “está listo” y se presentará la próxima semana 

@clarincom Que mesita de ideólogos!! El pasante @Martin_M_Guzman  que nos está fundiendo, un narco que no trabajo en su puta vida, el imbécil inútil de @CarlosHeller, e imagínense que vendría a ser el que los encabeza @alferdez !! Alguno que sirva para algo? {'jmperez': False, 'mkondra': True}

Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios” 

@clarincom @dlipo77 Por lo menos Gine bra admite que está en bolas {'jmperez': False, 'mkondra': True}

Verónica Magario pidió endurecer la cuarentena: “D

In [67]:
show_from_df(non_hateful_comments)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION No la escuche hablar del apriete de moyano a ML o del festejo en quilmes.
@LANACION Quédate encerrada en tu casa sin trabajar hasta que ( si es que no recibís sobres) te cagues de hambre y te hundas profundamente en una quiebra irreversible
@LANACION Siii!!!
Y a quien le importa lo que a vos te pareció?
@LANACION ES IRRESPONSABLE LUCHAR POR LA REPÚBLICA Y TERMINAR CON ESTA FARSA!!?
@LANACION Quien???
@LANACION Imaginarme hablando bien de este gobierno me resulta totalmente imposible... Escuchar viejos audios del presidente y escucharlo en la actualidad es un shock a las neuronas... Como hace señora ?? su confusión con "marcha anticuarentena" sugiere al menos desinformacion
@LANACION No te escuché decir lo mismo cuando los Moyano bloquearon Mercado Libre.!!!
O con todas las marchas y protestas qué hay en la 9 de Julio.
Sos poco creíble con tu discurso oficialista.!!!
@LANACION Seguro le “da 

## OTROS

In [68]:
agree_comments, disagree_comments, non_hateful_comments = get_agrees_and_disagrees(
    df_labels, "OTROS", users
)

In [69]:
show_from_df(agree_comments)

In [70]:
show_from_df(disagree_comments)


Ginés González García admite que prohibieron a los runners por una cuestión de imagen “más que por los contagios” 

@clarincom En cualquier país normal, a este impresentable sorete ya lo hubieran rajado.

Verónica Magario pidió endurecer la cuarentena: “Debemos frenar la circulación de personas en el AMBA para evitar contagios” 

@infobae Enserrate vos en una jaula.  Lpmqtp


## LLAMADO A LA ACCIÓN

Queremos ver:

- Aquellas que hayan sido etiquetadas como odiosas


In [41]:
my_users = ['amoro', 'jmperez', 'jzajac', 'mkondra', 'ndebandi']
df_calls_for_action = pd.DataFrame({"name": my_users})
df_calls_for_action.set_index("name", inplace=True)

for comment_label in CommentLabel.objects.filter(is_hateful=True, article_label__user__username__in=my_users):
    username = comment_label.article_label.user.username
    df_calls_for_action.loc[username, comment_label.comment_id] = comment_label.calls_for_action

In [42]:
kappa_of(df_calls_for_action, ['amoro', 'jmperez', 'jzajac', 'mkondra', 'ndebandi'])

(0.266284257751689, 36)

In [43]:
for i in range(len(df_calls_for_action)):
    user1 = df_calls_for_action.index[i]
    for j in range(i+1, len(df_calls_for_action)):
        user2 = df_calls_for_action.index[j]
        
        print(f"Kappa {user1:<10} - {user2:<10} = {kappa_of(df_calls_for_action, [user1, user2])}")

Kappa amoro      - jmperez    = (0.1608876560332872, 55)
Kappa amoro      - jzajac     = (0.339597315436242, 82)
Kappa amoro      - mkondra    = (-0.04819277108433673, 58)
Kappa amoro      - ndebandi   = (0.2285714285714288, 60)
Kappa jmperez    - jzajac     = (0.450472667488697, 191)
Kappa jmperez    - mkondra    = (0.16836262719703984, 124)
Kappa jmperez    - ndebandi   = (0.2723404255319149, 57)
Kappa jzajac     - mkondra    = (0.36821192052980173, 159)
Kappa jzajac     - ndebandi   = (0.5074235807860265, 94)
Kappa mkondra    - ndebandi   = (0.2587859424920122, 58)


In [44]:
kappa_of(df_calls_for_action, ['mkondra', 'ndebandi'])

(0.2587859424920122, 58)

In [45]:
kappa_of(df_calls_for_action, ['jzajac', 'mkondra',])

(0.36821192052980173, 159)

Bajo, bajo... aunque podría ser peor

¿Sobre todos los comentarios etiquetados, qué proporción "llaman a la acción"?

In [46]:
df_calls_for_action.sum(axis=1) / df_calls_for_action.notna().sum(axis=1)

name
amoro       0.059524
jmperez     0.191176
jzajac      0.108108
mkondra     0.030303
ndebandi    0.104167
dtype: float64

In [47]:
df = no_null_columns(df_calls_for_action.loc[["jzajac", "mkondra"]])

df.sum(axis=1)

name
jzajac     20.0
mkondra     5.0
dtype: float64

In [48]:

agree_comments = df[df.columns[df.all()]]
agree_comments.shape

no_agree = ~((df.sum() == 0) | (df.sum() == len(users)))
no_agree_comments = df[df.columns[no_agree]]

In [49]:
show_from_df(agree_comments)


El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

@infobae Napal y tema solucionado
@infobae Pero si tienen k morir todos los presos

Coronavirus en la Argentina: Hebe de Bonafini pidió suspender la marcha del 24 de marzo 

@clarincom Nuestra oportunidad de matar esta bosta..
@clarincom Algun infectado que le tosa en la cara a esta mujer please.

Condenan a ocho años de cárcel por corrupción al ex presidente ecuatoriano Rafael Correa 

@clarincom Pena de muerte deberían darle a ese enfermo mental comunista


In [51]:
show_from_df(no_agree_comments, show=False)


Cecilia Roth: "La marcha anticuarentena me pareció una irresponsabilidad sanitaria" 

@LANACION Esta señora grupo de riesgo debería cuidarse de no excederse con la fafafa en vez de usar el dedito acusador.
@LANACION Otra imbesil kaka! Anda hacerte ortear!

El primer muerto en los motines de las cárceles de Santa Fe fue un barra de Colón 

@infobae Que no pare el fuego y rociarlo con nafta es una buena receta..
@infobae No se pierde nada son una mierda . Todos los que están presos... que mueran todos
@infobae Esperemos que sea el primero pero no el último
@infobae Solo Me preocupa por los gastos q ocasionaron. Hay q ir con todo
@infobae @Llucia880955901 Pq no dejan que se maten entre ellos,son lacras que no sirven para nada, hubieran aprovechado y limpiaban todo de esta mierda

Cecilia Roth: “La marcha anticuarentena me pareció una irresponsabilidad sanitaria” 

@infobae Hacete un enema..boluda..actriz de 4ta..robando siempre..vos sos una irresponsabilidad enfermante

Juntos por el Cam